In [1]:
from lxml import etree as ET
import pandas as pd
import os
from xml_utils import xml_pretty_print
import numpy as np

# Import DLC database as a template

In [3]:
dlc_fname = 'CollectedData_Charlie.h5'

df_original = pd.read_hdf(dlc_fname)
df_original

scorer                                   Charlie                        \
bodyparts                                neuron0        neuron1          
coords                                         x      y       x      y   
labeled-data/test_100frames.ome/img0.tif   263.0  319.0   297.0  318.0   
labeled-data/test_100frames.ome/img1.tif   364.0  416.0   382.0  381.0   
labeled-data/test_100frames.ome/img2.tif   423.0  471.0   446.0  438.0   
labeled-data/test_100frames.ome/img3.tif   387.0  462.0   419.0  440.0   
labeled-data/test_100frames.ome/img4.tif   316.0  401.0   357.0  409.0   

scorer                                                                  \
bodyparts                                neuron2        neuron3          
coords                                         x      y       x      y   
labeled-data/test_100frames.ome/img0.tif   283.0  301.0   249.0  326.0   
labeled-data/test_100frames.ome/img1.tif   366.0  386.0   362.0  429.0   
labeled-data/test_100frames.ome/img2.tif   428.0  444.0   419.0  485.0   
labeled-data/test_100frames.ome/img3.tif   404.0  440.0   375.0  473.0   
labeled-data/test_100frames.ome/img4.tif   345.0  394.0   300.0  404.0   

scorer                                                   ...                 \
bodyparts                                neuron4         ... neuron9          
coords                                         x      y  ...       x      y   
labeled-data/test_100frames.ome/img0.tif   220.0  347.0  ...   272.0  272.0   
labeled-data/test_100frames.ome/img1.tif   358.0  466.0  ...   334.0  371.0   
labeled-data/test_100frames.ome/img2.tif   407.0  521.0  ...   398.0  423.0   
labeled-data/test_100frames.ome/img3.tif   349.0  499.0  ...   379.0  412.0   
labeled-data/test_100frames.ome/img4.tif   265.0  409.0  ...   345.0  360.0   

scorer                                                                    \
bodyparts                                neuron10        neuron11          
coords                                          x      y        x      y   
labeled-data/test_100frames.ome/img0.tif    278.0  264.0    338.0  275.0   
labeled-data/test_100frames.ome/img1.tif    332.0  361.0    389.0  329.0   
labeled-data/test_100frames.ome/img2.tif    394.0  415.0    442.0  371.0   
labeled-data/test_100frames.ome/img3.tif    378.0  402.0    436.0  375.0   
labeled-data/test_100frames.ome/img4.tif    351.0  352.0    411.0  353.0   

scorer                                                                    
bodyparts                                neuron12        neuron13         
coords                                          x      y        x      y  
labeled-data/test_100frames.ome/img0.tif    439.0  244.0    441.0  175.0  
labeled-data/test_100frames.ome/img1.tif    464.0  246.0    422.0  196.0  
labeled-data/test_100frames.ome/img2.tif    458.0  249.0    395.0  241.0  
labeled-data/test_100frames.ome/img3.tif    463.0  259.0    402.0  238.0  
labeled-data/test_100frames.ome/img4.tif    480.0  244.0    426.0  212.0  

[5 rows x 28 columns]

In [4]:
df_original = df_original.sort_index()
all_files = df_original.index
print(all_files)

Index(['labeled-data/test_100frames.ome/img0.tif',
       'labeled-data/test_100frames.ome/img1.tif',
       'labeled-data/test_100frames.ome/img2.tif',
       'labeled-data/test_100frames.ome/img3.tif',
       'labeled-data/test_100frames.ome/img4.tif'],
      dtype='object')


# Import XML from Icy

In [5]:
# fname = os.path.join('few_frame_videos', 'icy_40neurons_tracked.xml')
fname = os.path.join('few_frame_videos', 'icy_14neurons_slice13.xml')
et_icy = ET.parse(fname)

In [6]:
et2 = et_icy.getroot()
num_neurons = len(et2) - 2
print("Found {} neurons".format(num_neurons))

Found 14 neurons


# Write dataframe in DLC format

In [7]:
save_z_coordinate = False
using_original_fnames = False

In [15]:
scorer = df_original.columns.levels[0][0]

if save_z_coordinate:
    coord_names = ['x', 'y', 'z']
else:
    coord_names = ['x', 'y']

if using_original_fnames:
    relativeimagenames=df_original.index
else:
    relativeimagenames = ['labeled-data/test_100frames_13slice/img00.png',
       'labeled-data/test_100frames_13slice/img01.png',
       'labeled-data/test_100frames_13slice/img02.png',
       'labeled-data/test_100frames_13slice/img03.png',
       'labeled-data/test_100frames_13slice/img04.png']
print(relativeimagenames)
print("Assumes filenames in the DLC annotation are same as the Icy tracker, after alphabetizing")

output_path = '.'

coords = np.empty((len(relativeimagenames),len(coord_names),))

dataFrame = None
i_neuron_name = 0

['labeled-data/test_100frames_13slice/img00.png', 'labeled-data/test_100frames_13slice/img01.png', 'labeled-data/test_100frames_13slice/img02.png', 'labeled-data/test_100frames_13slice/img03.png', 'labeled-data/test_100frames_13slice/img04.png']
Assumes filenames in the DLC annotation are same as the Icy tracker, after alphabetizing


In [16]:

# Build correctly DLC-formatted dataframe
for i_neuron in range(num_neurons):
    bodypart = 'neuron{}'.format(i_neuron_name)
    
    i_xml = i_neuron + 1 # The first entry in the xml file is the 'trackfile' class
    this_detections = et2[i_xml]
    if len(this_detections) == 0:
        print("No tracks found for entry {}; continuing".format(i_neuron))
        continue
    elif len(this_detections) > 1:
        error("More than one track found in this group; formatting not supported; aborting")
    else:
        this_detections = this_detections[0]
        i_neuron_name = i_neuron_name + 1

    # Get xyz or xy coordinates for one neuron, for all files
    for i2 in range(len(relativeimagenames)):
        this_track = this_detections[i2]
        if save_z_coordinate:
            coords[i2,:] = np.array([int(float(this_track.get('x'))), 
                                     int(float(this_track.get('y'))), 
                                     int(float(this_track.get('z'))) ])
        else:
            coords[i2,:] = np.array([int(float(this_track.get('x'))), 
                                     int(float(this_track.get('y')))])
                

    index = pd.MultiIndex.from_product([[scorer], [bodypart],
                                        coord_names],
                                        names=['scorer', 'bodyparts', 'coords'])

    frame = pd.DataFrame(coords, columns = index, index = relativeimagenames)
    dataFrame = pd.concat([dataFrame, frame],axis=1)


dataFrame.to_csv(os.path.join(output_path,"CollectedData_" + scorer + ".csv"))
dataFrame.to_hdf(os.path.join(output_path,"CollectedData_" + scorer + '.h5'),'df_with_missing',format='table', mode='w')

    
print("Finished")

Finished


# Scratch

In [8]:
i = 2
print(type(et2[i][0]))
print(len(et2[i][0]))
et2[i][0][1].get('x')
et2[i][0][0].get('x')

<class 'lxml.etree._Element'>
5


'297.6748300071666'

In [8]:
xml_pretty_print(et2[2], 1)

track
{'id': '-744387789'}


In [9]:
len(et2[1])

0

'Charlie'